VERSION 4 changes logic to first weight making lsd shortest, then min available squares
the hope is that this allows for more nuances strategy, like covering both diagonals and leaving a vertical/horizontal open
in the hopes of forcing to a shorter diagonal position

- also fixed a bug in calculating shortest diagonal

In [ ]:
from sage.all import graphs
import copy, sys, math

def available_squares(cop_pos: list, robber_pos: tuple) -> list:
    #Function that returns the set of squares that are available to the robber
    #return the adjacent vertices of robber_pos \ adjacent vertices of cops    
    c_neighbors = set(cop_pos)
    for cop in cop_pos:
        c_neighbors = c_neighbors.union(set(G.neighbors(cop)))

    if robber_pos == -1: 
        return set(G.vertices()) - c_neighbors
    else:
        r_neighbors = set(G.neighbors(robber_pos)).union({robber_pos})
        return r_neighbors - c_neighbors

def get_axis(cop_pos, robber_pos):
    #returns 'h', 'v', 'posd', 'negd' or none for axis which a cop position occupies
    #assuming given correct input ie intersecting but not same posn
    if cop_pos == robber_pos:
        return 'h'
        
    if robber_pos[0] == cop_pos[0]:
        return 'v'
    elif robber_pos[1] == cop_pos[1]:
        return 'h'
    elif cop_pos[0] + cop_pos[1] == robber_pos[0] + robber_pos[1]:
        return 'negd'
    elif cop_pos[0] - cop_pos[1] == robber_pos[0] - robber_pos[1]:
        return 'posd'
    else:
        return None

def remove_axes_squares(robber_pos, avail, occupied_axes):
    #filters out the squares which are on a cop occupied axis from a list of vertices
    for axis in map(lambda a: a[0], set(filter(lambda a: a[1], occupied_axes.items()))):
        if axis == 'h':
            #remove all with move[1] == robber_pos[1]
            avail = set(filter(lambda posn: posn[1] != robber_pos[1], avail))
        elif axis == 'v':
            #remove all with move[0] == robber_pos[0]
            avail = set(filter(lambda posn: posn[0] != robber_pos[0], avail))
        elif axis == 'negd':
            #remove all w/move[0]+move[1] == robber_pos[0]+robber_pos[1]
            avail = set(filter(lambda posn: posn[0] + posn[1] != robber_pos[0] + robber_pos[1], avail))
        elif axis == 'posd':
            #remove all w/move[0]-move[1] == robber_pos[0]-robber_pos[1]
            avail = set(filter(lambda posn: posn[0] - posn[1] != robber_pos[0] - robber_pos[1], avail))
    return avail

def get_intersecting_squares(cop_pos, robber_pos, occupied_axes):
    #compiles a list of the squares which cops can reach that directly attack the robber on a unique line
    avail = set(G.neighbors(cop_pos)).union({cop_pos}).intersection(
        set(G.neighbors(robber_pos)).union({robber_pos}))
    return remove_axes_squares(robber_pos, avail, occupied_axes)

def get_SD_length(robber_pos):
    #determines the length of the SD, specifically for queen graph
    r0, c0 = robber_pos
    pos_diag = [v for v in G.vertices() if (v[0] - v[1]) == (r0 - c0)]
    neg_diag = [v for v in G.vertices() if (v[0] + v[1]) == (r0 + c0)]
    #print(f"for {robber_pos}, posd len = {len(pos_diag)}, negd len = {len(neg_diag)}")

    return min(len(pos_diag), len(neg_diag))

def get_min_LSD_move(robber_pos: tuple, moves: list) -> tuple:
    #determines the cop config which forces the minimum LSD
    min_LSD = sys.maxsize
    min_move = tuple()

    for move in moves:
        r_moves = available_squares(move[0], robber_pos)
        LSD = -1

        #get LSD of robber for this possible move
        for r_move in r_moves:
            SD_len = get_SD_length(r_move)
            if SD_len > LSD:
                LSD = SD_len
    
    if LSD < min_LSD:
        min_LSD = LSD
        min_move = move
    
    return min_move

def seenp(move: list, robber_pos: tuple, cop_states: list, robber_states: list)-> bool:
    if not (cop_states and robber_states): #empty states given for some reason
        return False
    
    for t in range(len(cop_states) - 1, -1, -1):
        if (set(cop_states[t]) == set(move) and robber_states[t] == robber_pos):
            print("SEEN")
            return True
        
    return False

#n^2 algo where n = # available moves, which is constant...
def remove_seen_moves(moves: list, robber_pos: tuple, cop_states: list, robber_states: list)-> list:
    #removes cop moves from a list of available cop moves if that state has already been seen

    if (len(cop_states) != len(robber_states)):
        raise Exception("given nonmatching states")
    

    new_moves = [(m,v) for (m,v) in moves
             if not seenp(m, robber_pos, cop_states, robber_states)]

    return new_moves

def get_closest_unoccupied(cop_pos: list, robber_pos: tuple, idx: int)-> tuple:
    V = G.neighbors(cop_pos[idx])

    min_dist = sys.maxsize
    min_pos = cop_pos[idx]
    
    for v in V:
        dist = math.sqrt((v[0] - robber_pos[0])**2 + (v[1] - robber_pos[1])**2)
        if dist < min_dist and v not in cop_pos:
            min_dist = dist
            min_pos = v

    return min_pos

def get_SD_matrix(G):
    #short diagonal matrix for queen graphs
    n = int(math.sqrt(len(G.vertices())))
    arr = [[0 for _ in range(n)] for _ in range(n)]

    #note that sagemath graph coords start from bottom left
    for r in range(n):
        for c in range(n):
            #print((r, c))
            arr[r][c] = get_SD_length((r, c))

    return arr

def get_LSD_len(SDs, robber_pos, cop_pos):
    '''
    determine the longest short diagonal out of all available moves given state
    '''
    avail = available_squares(cop_pos, robber_pos)
    LSD_len = -1

    for move in avail:
        curr_sd = SDs[move[0]][move[1]] 
        #print(f"move: {move}, curr_sd: {curr_sd}")
        if curr_sd > LSD_len:
            #print("hi")
            LSD_len = curr_sd
    
    #print(LSD_len)
    return LSD_len

def minimize_avail_helper(curr_cop_pos, robber_pos, i, occupied_axes, cop_moves, robber_moves, SDs):
    #goal is to find the minimizing config of cops
    #so track a min config and min robber avail squares
    #occupied axes is a dict, represents which robber axes are occupied in current backtracking iteration

    #base case i > #cops
    if i >= len(curr_cop_pos):
        return curr_cop_pos, get_LSD_len(SDs, robber_pos, curr_cop_pos)
        #return curr_cop_pos, len(available_squares(curr_cop_pos, robber_pos))

    avail = get_intersecting_squares(curr_cop_pos[i], robber_pos, occupied_axes)
    moves = list() # list(posn) -> avail_squares

    for move in avail:
        #find axes this occupies
        axis = get_axis(move, robber_pos)
        occupied_axes[axis] = True
        new_cop_pos = copy.deepcopy(curr_cop_pos)
        new_cop_pos[i] = move #move cop i
        
        curr_config, curr_LSD = minimize_avail_helper(new_cop_pos, robber_pos, i+1, occupied_axes, cop_moves, robber_moves, SDs)
        moves.append((curr_config, curr_LSD))
    
        occupied_axes[axis] = False

    #remove all moves which revisit board states
    moves = remove_seen_moves(moves, robber_pos, cop_moves, robber_moves)

    if not moves:
        #animal case-- go closer to the robber
        curr_cop_pos[i] = get_closest_unoccupied(curr_cop_pos, robber_pos, i)
        return minimize_avail_helper(curr_cop_pos, robber_pos, i+1, occupied_axes, cop_moves, robber_moves)
    
    #get squares with min LSD for robber
    vals = map(lambda tup: tup[1], moves)
    min_val = min(vals)
    min_avail_moves = list(filter(lambda tup: tup[1] == min_val, moves))

    #sort by which has min # available squares for robber
    #best_move = get_min_LSD_move(robber_pos, min_avail_moves)
    best_move = sorted(min_avail_moves, 
                       key=lambda move: len(available_squares(move[0], robber_pos)))[0]
    
    return best_move[0], best_move[1]

In [90]:
def minimize_available(cop_pos: list, robber_pos: tuple, cop_states, robber_states, SDs) -> list:
    # Function that returns the move for the cops that minimizes the number of available squares for the robber
    #this function could be the combinatorially large one, but we are going to introduce our greedy heuristic
    #our strategy is such: the cops should always directly threaten a unique line of movement
    #get set of cop i available_moves \intersect set of robber 
    #filter out whichever are on occupied axes
    #use backtracking algorithm, recursively call min_avail_helper w/i+1, new cop_pos

    occupied_axes = {
        'h': False,
        'v': False,
        'negd': False,
        'posd': False
    }
    
    min_config, min_squares = minimize_avail_helper(cop_pos, robber_pos, 0, occupied_axes, cop_states, robber_states, SDs)
    
    return min_config

def maximize_available(cop_pos: list, cop_states, robber_states, SDs, robber_pos:tuple =-1) -> tuple: #-1 denotes no robber placed yet ie startin
    # Function that returns move for the robber that maximizes the number of squares for their next turn (assuming cops try to minimize)
    #get set of valid moves available_squares
    #for all moves m, call available_squares(cop_pos, m), get size of set
    #track max size and move, return that move
    #O(n)

    r_neighbors = available_squares(cop_pos, robber_pos)

    moves = dict() # move -> (available moves in anticipation, LSD length in anticipation)
    
    for move in r_neighbors:
        #Q: here, do we want cop moves to take into account that cops wont repeat moves ?
        #at this point, cops are making suboptimal moves
        # i guess dont take into account, as robber doesnt care for repeating moves?
        cop_response = minimize_available(cop_pos, move, [], [], SDs)
        max_min_val = len(available_squares(cop_response, move))
        longest_sd_len = get_LSD_len(SDs, move, cop_response)

        moves[move] = (max_min_val, longest_sd_len)
    
    if not moves: #no available moves, you lose!
        return robber_pos

    #sort by decreasing sd, then by decreasing available squares
    #recall values[1] is lsd length, values[0] is the available squares for key=move
    best_move = max(moves.items(), key=lambda item: (item[1][1], item[1][0]))
    print(best_move)
    return best_move[0]

def k_cop_win(cop_start, robber_start, itr, cop_states, robber_states, SDs):
    #returns true if cop win possible with k cops
    cop_move = minimize_available(cop_start, robber_start, cop_states, robber_states, SDs) # The cops try to minimize the available squares
    print("Cops move:", cop_move)
    cop_states.append(cop_move)
    robber_states.append(robber_start)
    avail_squares.append(len(available_squares(cop_move, robber_start)))
    robber_move = maximize_available(cop_move, cop_states, robber_states, SDs, robber_start) # The robber tries to maximize this minimum
    print("Robber moves to:", robber_move)
    print(avail_squares[-1], "squares available for after move", itr)

    cop_states.append(cop_move)
    robber_states.append(robber_move)
    
    # Checking if the cops have captured the robber
    if len(available_squares(cop_move, robber_move)) == 0:
        return True, cop_states, robber_states
    
    # If the cops can't decrease the number of available moves, they lose
    #if len(avail_squares) > 1 and avail_squares[-1] > avail_squares[-2]:
    #    print("available squares increased")
    #    return False

    if itr > 1000:
        print("iterations exceeded")
        return False, cop_states, robber_states
            
    # If the cop's haven't won yet, keep going
    return k_cop_win(cop_move, robber_move, itr+1, cop_states, robber_states, SDs)

In [13]:
#CODE FOR GENERATING ANIMAL/ROYAL GRAPHS GIVEN DIRECTIONS

def make_graph(n, slopes, animal=False):
    from sage.all import QQ, Infinity

    vertices = [(x, y) for x in range(n) for y in range(n)]
    G = Graph()
    G.add_vertices(vertices)

    for i, (x1, y1) in enumerate(vertices):
        # Convert slope list to exact rational numbers or Infinity
        D = set(QQ(s) if s != 'inf' else Infinity for s in slopes)
        for j in range(i+1, len(vertices)):
            x2, y2 = vertices[j]
            dx = x2 - x1
            dy = y2 - y1

            if dx == 0:
                slope = Infinity
            else:
                slope = QQ(dy) / QQ(dx)

            if slope in D:
                G.add_edge((x1, y1), (x2, y2))
                if animal:
                    D.remove(slope)

    return G


In [91]:
'''
EDIT THIS CODE TO CHANGE THE GRAPH
similar to evans code, just input slopes into a list and pass it into make_graph function (specify animal or royal w/bool)
'''

n=19
knight = [2, -2, 1/2, -1/2]
queen = [0, 'inf', 1, -1]
bishop = [1, -1]
idk = [1/3, -1/3, 3, 3]
G = make_graph(n, queen, False)
SDs = get_SD_matrix(G)
#print(SDs)


In [92]:
#n = 15
#T = n**2
#G = graphs.QueenGraph([n,n])

'''
run this code to run the above greedy algorithm on the graph defined above
define a list of tuples representing where you want your cops to start in (x,y) coords
then pass into play_game()

you could also use this to iteratively check the largest n for which k cops can win w/this algorithm in a loop
'''

avail_squares = list()

def play_game(cops_start):
    robber_start = maximize_available(cops_start, [], [], SDs)
    print(f"rstart: {robber_start}, cops: {cops_start}")
    robber_moves = [robber_start]
    cop_moves = [cops_start]
    return k_cop_win(cops_start, robber_start, 1, cop_moves, robber_moves, SDs)

#6x6 domination
dom_start = math.floor((n+1)/2) - 3
dom_set = [(dom_start, dom_start), (dom_start + 4, dom_start + 2), (dom_start + 2, dom_start + 4)]

corner_start = [(0,0), (n-1,n-1), (0,n-1)]

two_cops = [(0,0), (n-1,n-1)]

mid = math.floor(n/2)
four_cops = [(mid, mid), (mid-1, mid), (mid-1,mid-1), (mid,mid-1)]


winp, cop_moves, robber_moves = play_game(dom_set)
print("Cop win:", winp)
print(cop_moves, robber_moves)

((5, 10), (12, 13))
rstart: (5, 10), cops: [(7, 7), (11, 9), (9, 11)]
Cops move: [(2, 7), (6, 9), (9, 10)]
((5, 11), (12, 12))
Robber moves to: (5, 11)
12 squares available for after move 1
Cops move: [(9, 7), (8, 11), (4, 10)]
((5, 6), (5, 12))
Robber moves to: (5, 6)
12 squares available for after move 2
Cops move: [(9, 6), (10, 11), (5, 10)]
((6, 5), (5, 12))
Robber moves to: (6, 5)
5 squares available for after move 3
Cops move: [(10, 5), (11, 10), (6, 11)]
((7, 4), (5, 12))
Robber moves to: (7, 4)
5 squares available for after move 4
Cops move: [(8, 5), (11, 4), (7, 10)]
((5, 6), (5, 12))
Robber moves to: (5, 6)
5 squares available for after move 5
Cops move: [(5, 8), (9, 6), (8, 9)]
((6, 5), (5, 12))
Robber moves to: (6, 5)
5 squares available for after move 6
Cops move: [(5, 4), (10, 5), (6, 11)]
((4, 7), (5, 12))
Robber moves to: (4, 7)
5 squares available for after move 7
Cops move: [(2, 7), (4, 5), (7, 10)]
((8, 3), (5, 12))
Robber moves to: (8, 3)
5 squares available for aft

In [80]:
from sage.all import graphs
import networkx as nx
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from IPython.display import display, Javascript
import ipywidgets as widgets
import math

def get_state(r_state, c_state):
    #get dict w/red = occupied by cops, blue = cops, black = robber, green = available for robber movement
    cop_occ = set()
    for cop in c_state:
        cop_occ = cop_occ.union(set(G.neighbors(cop)))

    cop_occ -= set(c_state)
    
    state = {
        'blue': set(c_state),
        'black': {r_state},
        'green': set(available_squares(c_state, r_state)) - {r_state},
        'red': cop_occ - {r_state}
    }
    return state

def convert_to_state(robber_moves, cop_moves):
    if len(robber_moves) != len(cop_moves):
        raise Exception("nonequal lists given")
    
    states = list()

    for state in range(len(robber_moves)):
        states.append(get_state(robber_moves[state], cop_moves[state]))

    return states

# Update function for each turn
def update(turn):
    fig, ax = plt.subplots(figsize=(12, 12))
    nx.draw(nx_G, pos, ax=ax, node_color='lightgrey', node_size=400, with_labels=False)
    
    for color, nodes in states[turn].items():
        nx.draw_networkx_nodes(nx_G, pos, nodelist=list(nodes), node_color=color, node_size=400, ax=ax)
    
    ax.set_title(f"Turn {math.floor(turn / 2) + 1}")
    ax.set_axis_off()
    plt.show()


In [ ]:
from ipyevents import Event

'''
run this code to visualize moves made
'''

# Build Sage Queen Graph
pos = { (i, j): (i, j) for i in range(n) for j in range(n) }
G.set_pos(pos)
nx_G = G.networkx_graph()

# game states 
#list of dicts<string color -> set of posns
states = convert_to_state(robber_moves, cop_moves)
#print(states)

# Create the slider and output widget
slider = widgets.IntSlider(min=0, max=len(states)-1, step=1, value=0)
out = widgets.interactive_output(update, {'turn': slider})

# Create the event handler
event = Event(source=slider, watched_events=['keydown'])

def handle_event(event):
    if event['key'] == 'ArrowRight':
        slider.value = min(slider.max, slider.value + slider.step)
    elif event['key'] == 'ArrowLeft':
        slider.value = max(slider.min, slider.value - slider.step)

event.on_dom_event(handle_event)

# Display everything
display(slider, out)

IntSlider(value=0, max=302)

Output()

In [189]:
import unittest
from sage.graphs.graph_generators import graphs

G = graphs.QueenGraph([4, 4])
#r = (3, 2)

#w/4x4
class TestAlg(unittest.TestCase):            
    def test_get_axis(self):
        self.assertEqual(get_axis((1, 2), r), 'h')
        self.assertEqual(get_axis((5, 4), r), 'posd')
        self.assertEqual(get_axis((3, 6), r), 'v')
        self.assertEqual(get_axis((1, 4), r), 'negd')
        self.assertIsNone(get_axis((4, 4), r))

    def test_remove_axes_squares_basic(self):
        occupied_axes = {
            'h': True,
            'v': False,
            'negd': False,
            'posd': False
        }
        avail = set(G.neighbors((0, 0))).union({(0, 0)})
        avail_axes_removed = list(remove_axes_squares(r, avail, occupied_axes))
        self.assertEqual(len(avail), len(avail_axes_removed) + 2)
        self.assertEqual(set(avail) - set(avail_axes_removed), {(0, 2), (2, 2)})

    def test_remove_axes_squares_more_axes(self):
        occupied_axes = {
            'h': True,
            'v': True,
            'negd': False,
            'posd': False
        }
        avail = set(G.neighbors((0, 0))).union({(0, 0)})
        avail_axes_removed = list(remove_axes_squares(r, avail, occupied_axes))
        self.assertEqual(len(avail), len(avail_axes_removed) + 4)
        self.assertEqual(set(avail) - set(avail_axes_removed), {(0, 2), (2, 2), (3, 0), (3, 3)})

        occupied_axes['posd'] = True
        avail_axes_removed = list(remove_axes_squares(r, avail, occupied_axes))
        self.assertEqual(len(avail), len(avail_axes_removed) + 5)
        self.assertEqual(set(avail) - set(avail_axes_removed), {(0, 2), (2, 2), (3, 0), (3, 3), (1, 0)})

        occupied_axes['negd'] = True
        avail_axes_removed = list(remove_axes_squares(r, avail, occupied_axes))
        self.assertEqual(len(avail), len(avail_axes_removed) + 5)
        self.assertEqual(set(avail) - set(avail_axes_removed), {(0, 2), (2, 2), (3, 0), (3, 3), (1, 0)})

    def test_get_intersecting_squares(self):
        occupied_axes = {
            'h': False,
            'v': False,
            'negd': False,
            'posd': False
        }
        result = get_intersecting_squares((0, 0), r, occupied_axes)
        self.assertEqual(result, {(3, 0), (0, 2), (3, 3), (2, 2), (1, 0)})

        occupied_axes['h'] = True
        result = get_intersecting_squares((0, 3), r, occupied_axes)
        self.assertEqual(sorted(result), [(2, 1), (2, 3), (3, 0), (3, 3)])

    def test_get_SD_length(self):
        self.assertEqual(get_SD_length((0,0)), 1)
        self.assertEqual(get_SD_length((1,0)), 2)
        self.assertEqual(get_SD_length((0,1)), 2)
        self.assertEqual(get_SD_length((1,1)), 3)



G = graphs.QueenGraph([6,6])

#w/6x6
class TestAlg2(unittest.TestCase):
    def test_get_min_LSD_move(self):
        self.assertEqual(get_min_LSD_move((2,3),[([(4,3), (2,4)], 1), ([(4,3),(4,5)], 1)]), ([(4,3),(4,5)], 1))
        
    def test_maximize_avail(self):
        self.assertEqual(maximize_available([(0,0), (5,5)]),(2,3)) #stays in the center
        self.assertEqual(maximize_available([(2,2), (3,3)]),(1,4))
        
    def test_minimize_avail_helper(self):
        self.assertEqual(minimize_available([(1,0),(3,5)],(2,3), [], []), [(1,3),(0,5)]) #80% sure this is optimal
    
    def test_remove_seen(self):
        #test standard seen move
        self.assertEqual(remove_seen_moves([([(1,2),(3,4)], 1), ([(5,6),(7,8)], 2)], 
                          (1,2), [[(1,2),(3,4)], [(1,3),(3,4)]], [(1,2), (1,2)]), [([(5,6),(7,8)], 2)])
        #test out of order cop move -> should still be seen
        self.assertEqual(remove_seen_moves([([(3,4),(1,2)], 1), ([(5,6),(7,8)], 2)], 
                          (1,2), [[(1,2),(3,4)], [(1,3),(3,4)]], [(1,2), (1,2)]), [([(5,6),(7,8)], 2)])
        #test nonmatching curr robber pos
        self.assertEqual(remove_seen_moves([([(1,2),(3,4)], 1), ([(5,6),(7,8)], 2)], 
                          (1,3), [[(1,2),(3,4)], [(1,3),(3,4)]], [(1,2), (1,2)]), [([(1,2),(3,4)], 1), ([(5,6),(7,8)], 2)])     
# Run tests in Jupyter
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestAlg2))

.E..
ERROR: test_maximize_avail (__main__.TestAlg2.test_maximize_avail)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_9321/2615096988.py", line 80, in test_maximize_avail
    self.assertEqual(maximize_available([(Integer(0),Integer(0)), (Integer(5),Integer(5))]),(Integer(2),Integer(3))) #stays in the center
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: maximize_available() missing 2 required positional arguments: 'cop_states' and 'robber_states'

----------------------------------------------------------------------
Ran 4 tests in 0.010s

FAILED (errors=1)


SEEN
SEEN


<unittest.runner.TextTestResult run=4 errors=1 failures=0>